In [1]:
import numpy as np

In [2]:
import scipy.io as sio 


In [3]:
data = sio.loadmat("./data/2016-10-26_1/Combo3_V1.mat")
data['data']

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [ ]:
"""
data structure example
"""
data_c = data['data']
# ROI data
coordinates = data_c[0][0][0]
# imgPara
imgPara = data_c[0][0][1]
stim_type = imgPara['stim_type']
stim_time = imgPara['stim_time']
updatefr = imgPara['updatefr']
intertime = imgPara['intertime']
stimrep = imgPara['stimrep']
print stimrep
dt = imgPara['dt']
F = imgPara['F']
F_gray = imgPara['F_gray']
# spiketrain
spiketrain = data_c[0][0][2]
# number of neurons
spiketrain.shape[1]
# the spiketrain _st of cell 80
spiketrain_st = spiketrain[0,79][0] # shape=(20,3)
spiketrain_st_gray = spiketrain[0,79][1] # shape=(20,3)
# the spiketrain_st of cell 80, trial 13, stim_type = 2
spiketrain_st_trial_stimType = spiketrain[0,79][0][12,1]
# the specific spike timing at [0,i]
# print spiketrain_st_trial_stimType[0,1]

In [ ]:
print spiketrain[0,2][0][0,0]


In [6]:
def get_response_mat(spiketrain, imgPara, stimType, goodCells, plotRaster):
    
    numNeuron = len(goodCells) # the number of neurons
    print numNeuron
    
    # imgPara.stim_time = 32s, imgPara.dt = 0.075103, 
    # numFramesPerStim is the number of the frames within 32s movie stimulus
    
    numFramesPerStim = int(round(imgPara['stim_time'] / imgPara['dt']))
    # print numFramesPerStim
    spikeMat = []
    ## generate the spike timing for all the neurons through all trials
    for rep in range(imgPara['stimrep']):    
        spikesCurrentTrial = np.zeros((numNeuron, numFramesPerStim))
        spikesRaster = []
        cellI = 0
        for i in range(len(goodCells)):
            # spikesI: spiking timing of a specific neuron at a specific trial
            # print i
            spikesI = spiketrain[0,i][0][rep,stimType]
            # print spikesI
            spikesI = np.round(spikesI[np.nonzero(spikesI<=numFramesPerStim)])
            #print spikesI
            spikesI = spikesI[np.nonzero(spikesI>0)];
            spikesI = spikesI.astype(int)
            
            spikesI = spikesI - 1
            # print spikesI
            # along the 426 frames, spike timings was labeled
            spikesCurrentTrial[cellI,spikesI] = 1
            cellI  = cellI +1;
            spikesRaster.append(spikesI*imgPara['dt'] -1)
        

        # return spikeMat as the spiking time for all neurons
        spikeMat.append(spikesCurrentTrial)  
    
    # change spikeMat to be numpy array
    spikeMat = np.array(spikeMat)   
    print spikeMat.shape
    return spikeMat
            

In [7]:
def calculate_sparsity_over_trials(spikeMat, imgPara):
    print spikeMat.shape
    stimrep = int(imgPara['stimrep'])
    sparsity = np.zeros((stimrep,1))
    numFramesPerStim = int(round(imgPara['stim_time']/imgPara['dt']))
    # the returned sparsity is everaged for all the neurons' 
    # firings within each specific trial
    for rep in range(stimrep):
        numFramesArray = np.array(range(numFramesPerStim))
        spikesCurrentTrial = spikeMat[rep]  
        sparsity[rep] = np.mean(spikesCurrentTrial) # average by columns and by rows    
    return sparsity

In [8]:
dataFolderList = ['data/2016-10-26_1','data/2016-07-27_2','data/2016-06-21_1','data/2016-07-07_1'
            ,'data/2016-07-27_1','data/2016-08-09_1','data/2016-08-16_1']

In [9]:
area = 'V1'
for exp in range(len(dataFolderList)):
    print './'+dataFolderList[exp]+'/Combo3_'+area+'.mat'

./data/2016-10-26_1/Combo3_V1.mat
./data/2016-07-27_2/Combo3_V1.mat
./data/2016-06-21_1/Combo3_V1.mat
./data/2016-07-07_1/Combo3_V1.mat
./data/2016-07-27_1/Combo3_V1.mat
./data/2016-08-09_1/Combo3_V1.mat
./data/2016-08-16_1/Combo3_V1.mat


In [10]:
# import get_response_mat
# import calculate_sparsity_over_trials

numActiveNeuron = 0;
numTotalNeuron = 0;

area = 'V1'; # area should be V1 or AL

In [12]:
import numpy as np
import scipy.io as sio 
import matplotlib.pyplot as plt
import get_response_mat
import calculate_sparsity_over_trials

dataFolderList = ['data/2016-10-26_1','data/2016-07-27_2','data/2016-06-21_1','data/2016-07-07_1'
            ,'data/2016-07-27_1','data/2016-08-09_1','data/2016-08-16_1']
numActiveNeuron = 0;
numTotalNeuron = 0;

area = 'V1'; # area should be V1 or AL

for stimType in range(3):
    for exp in range(len(dataFolderList)):
    #for exp in range(1):
        data=sio.loadmat('./'+dataFolderList[exp]+'/Combo3_'+area+'.mat')
        data_c = data['data']
        # imgPara
        imgPara = data_c[0][0][1]
        print imgPara
        stim_type = imgPara['stim_type']
        stim_time = imgPara['stim_time']
        updatefr = imgPara['updatefr']
        intertime = imgPara['intertime']
        stimrep = imgPara['stimrep']
        dt = imgPara['dt']
        F = imgPara['F']
        F_gray = imgPara['F_gray']
        # spiketrain
        spiketrain = data_c[0][0][2]
        # number of neurons
        numNeuron = spiketrain.shape[1]
        numFramesPerStim = int(round(stim_time / dt))
        
        gratingResponse = [] # never be used
        spikesPerNeuron = np.zeros((numNeuron,3))
        
        for i in range(numNeuron):
            for j in range(stim_type):
                numSpike = 0
                for rep in range(stimrep):
                    spikesI = spiketrain[0,i][0][rep,j]
                    numSpike = numSpike + len(spikesI[0]);
               
                spikesPerNeuron[i, j] = numSpike;
            
        #print spikesPerNeuron
        print "Number of cells: %d" % numNeuron
        # Population Response to Natural Stimuli
         # goodCells = range(numNeuron); # choose all the cells to be goodCells
        goodCells = np.nonzero(spikesPerNeuron[:,stimType]>3)
        print goodCells[0].shape
        spikeMat = get_response_mat(spiketrain, imgPara, stimType,goodCells[0], 0);
        
        # show sparsity over trials
        sparsity = calculate_sparsity_over_trials(spikeMat, imgPara)
        numActiveNeuron = numActiveNeuron + sparsity * len(goodCells) * numFramesPerStim
        numTotalNeuron = numTotalNeuron + len(goodCells)
        """
        # spiketrain data structure example
        # the spiketrain _st of cell 78
        spiketrain_st = spiketrain[0,79][0] # shape=(20,3)
        spiketrain_st_gray = spiketrain[0,79][1] # shape=(20,3)
        # the spiketrain_st of cell 79, trial 13, stim_type = 2
        spiketrain_st_trial_stimType = spiketrain[0,79][0][12,1]
        # the specific spike timing at [0,i]
        print spiketrain_st_trial_stimType[0,1]
        """
        plt.plot(sparsity,'k')
        plt.xlabel('trial #')
        plt.ylabel('sparseness')
        plt.show()

#     
#     h=figure(4);clf;
#     subplot(2,2,1);
#     plot(numActiveNeuron/numTotalNeuron/numFramesPerStim, '-o')
#     title(['Area ' area ' Stimulus ' num2str(stimType) ' n=' num2str(numTotalNeuron)] );
#     xlabel('Trial #');
#     ylabel('Sparseness');
#     print(h,'-dpdf', ['figures/sparsity/sparseness_over_time_stim_' num2str(stimType) '_area_' area  '.pdf']);


[[ (array([[3]], dtype=uint8), array([[32]], dtype=uint8), array([[ 0.01666667]]), array([[8]], dtype=uint8), array([[20]], dtype=uint8), array([[ 0.07508917]]), array([[426, 426, 426]], dtype=uint16), array([[106]], dtype=uint8))]]
Number of cells: 352
(214,)
214
(20, 214, 426)
(20, 214, 426)


In [ ]:
a = np.array([[1,2,3],[2,3,4]])
b=[]
b.append(a)
b.append(a)
# print b[1]
a = np.array([1,2,3])
a=np.array([])

print a-1

In [ ]:
a = np.zeros((2,3))
a[1, [2]]=1
b=[[2.0]]
b=np.array(b)
b=b.astype(int)
a[1,b]=1
print b